In [69]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pickle import dump,load
from sklearn.feature_selection import SelectKBest,mutual_info_regression
from sklearn.linear_model import LassoCV,LinearRegression
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

import heapq

load data

In [133]:
data = pd.read_csv('./data/full_data.csv', encoding='utf-16', sep='\t')
data.drop(columns=['Unnamed: 0','KLD热风风门开度（%）d.1'], inplace=True)
data.head()
data = data.iloc[int(len(data) * 0.5):, :]

In [135]:
# data.head()

In [139]:
target = 'KLD热风风门开度（%）d'
y = data['KLD热风风门开度（%）d']
x = data.drop(columns=['KLD热风风门开度（%）d'])
col_names = x.columns.to_list()

In [137]:
if 'KLD热风风门开度（%）d.1' in data.columns.to_list():
    print('yes')
if 'KLD热风风门开度（%）d' in data.columns.to_list():
    print('yes')

yes


Feature Selection

In [140]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.8,shuffle=True)

In [107]:
x_test.shape,y_test.shape

((16592, 39), (16592,))

In [108]:
# x_train.head()

In [141]:
feature_select_result_df = pd.DataFrame(data={'feature': col_names})
feature_select_result_df = feature_select_result_df.set_index('feature')

mutual_reg_result = mutual_info_regression(x_train, y_train)
# selector = SelectKBest(mutual_info_regression, k=25)
# selector.fit(x_train, y_train)
# cols = selector.get_support(indices=True)
# features_selected = []
# for i in cols:
#     features_selected.append(col_names[i])
mir_max_index = heapq.nlargest(25, range(len(mutual_reg_result)), mutual_reg_result.take)

In [142]:
estimator_lasso = LassoCV()
estimator_lasso.fit(x_train, y_train)
LassoCV_feature_list = estimator_lasso.coef_

LassoCV_max_index = heapq.nlargest(25, range(len(LassoCV_feature_list)), LassoCV_feature_list.take)

In [143]:
selector = RFE(LinearRegression(), n_features_to_select=25)
selector.fit(x_train, y_train)
rfelr_feature_list = selector.ranking_

rfelr_index = np.where(rfelr_feature_list == 1)[0].tolist()

In [112]:
# Wrapper 2  random_state=42, max_depth=5
selector = RFE(RandomForestRegressor() , n_features_to_select=25)
selector.fit(x_train, y_train)
rferfr_feature_list = selector.ranking_

rferfr_index = np.where(rferfr_feature_list == 1)[0].tolist()

In [144]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)
dtr_feature = model.feature_importances_

dtr_max_index = heapq.nlargest(25, range(len(dtr_feature)), dtr_feature.take)

In [145]:
l = np.zeros(len(col_names))
for i in dtr_max_index:
    l[i] += 1
# for i in rferfr_index:
#     l[i] += 1
for i in rfelr_index:
    l[i] += 1
for i in LassoCV_max_index:
    l[i] += 1
for i in mir_max_index:
    l[i] += 1
l

array([3., 3., 3., 3., 3., 2., 2., 1., 3., 3., 3., 3., 3., 2., 3., 3., 1.,
       1., 4., 3., 1., 1., 2., 1., 2., 2., 2., 3., 3., 4., 1., 0., 2., 0.,
       0., 0., 0., 2., 0., 2., 2., 2., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
       3., 0., 0., 1., 0., 1., 1., 0., 2., 0., 0., 1.])

In [124]:
list40 = []
for i in range(len(l)):
    if l[i] > 2:
        list40.append(col_names[i])
list40

['1区冷凝水温度（℃）d',
 '1区蒸汽压力（bar）d',
 '1区筒壁温度（℃）d',
 '2区冷凝水温度（℃）d',
 '2区蒸汽压力（bar）d',
 '2区筒壁温度（℃）d',
 'KLD1区蒸汽薄膜阀开度（%）d',
 'KLD1区蒸汽薄膜阀开度反馈值TIC1-Y1（%）D',
 'KLD2区蒸汽薄膜阀开度（%）d',
 'KLD2区蒸汽薄膜阀开度反馈值TIC3-Y22（%）D',
 'KLD排潮温度（℃）d',
 'KLD排潮风门开度反馈值PIC1-Y6（%）D',
 'KLD烘丝机管道风速差压传感器读数d',
 'KLD烘后温度（℃）d',
 '制丝A线烘丝段用蒸汽流量',
 '烘前叶丝累计量（kg）d',
 'KLD热风风门开度（%）d.1']

In [146]:
list65 = []
for i in range(len(l)):
    if l[i] > 2:
        list65.append(col_names[i])
len(list65),list65

(18,
 ['时间',
  '1区冷凝水温度（℃）d',
  '1区蒸汽压力（bar）d',
  '1区筒壁温度（℃）d',
  '2区冷凝水温度（℃）d',
  'KLD1区蒸汽薄膜阀开度（%）d',
  'KLD1区蒸汽薄膜阀开度反馈值TIC1-Y1（%）D',
  'KLD2区蒸汽薄膜阀开度（%）d',
  'KLD2区蒸汽薄膜阀开度反馈值TIC3-Y22（%）D',
  'KLDD烘丝分汽缸压力',
  'KLD一次减压后蒸汽压力（bar）d',
  'KLD排潮温度（℃）d',
  'KLD排潮风门开度反馈值PIC1-Y6（%）D',
  'KLD烘丝机管道风速差压传感器读数d',
  'KLD热风进筒温度（℃）d',
  'KLD热风风速（m/s）d',
  'KLD热风风门开度反馈值SIC1-Y7（%）D',
  '烘前叶丝累计量（kg）d'])

In [13]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train, y_train)
with open('model/scaler.pickle', 'wb') as f:
    dump(scaler, f)